In [1]:
# Selector 사용 추천 왜? 활용 방법이 많기 때문이다
# 지칭, 구조, 관계 - 가상 선택자 혹은 형제, 자식, 부모 계열을 찾아가는 것
# pseudo selector를 확인하면 selector 확인이 가능하다
# 태그이름, #아이디, .클래스, .클래스.클래스, 태그이름#아이디, #아이디.클래스
# (주의) 띄어쓰기 다른 의미, 띄어쓰기는 자손을 의미함
# 자식 >, 다음 형제 +, 다음 형제노드들 ~

# ex) span + a ==> a를 찾아야하는데 이전에 span이라는 얘가 있었어야함
# <p>
#     <div></div>
#     <span></span><a></a>
# </p>
# 이렇게 되어 있으면 span 의 a 를 찾을 수 있다

# ex) span ~ a
#<p>
#     <div><a></a></div>    ... 1
#     <span></span><a></a><a></a><a></a><a></a><a></a><a></a>   ... 2
# </p>
# 1번 거르고 2번만 가지고 온다

# 가상선택자
# :has, :is, :nth-child(같은 부모를 공유하는 자식들의 순선), :nth-of-type(같은 종류의 태그 중 순번)
# <a class="A">
# <a class="B">...
# ex) a:is(.A) class A를 가진 a태그 찾기

# Crawler - link를 따라 다니면서 웹페이지에 무슨 내용이 있었는지 수집(index/indexing) + 다음 link를 찾아서 또 따라다니는 역할
# Tree (bfs, dfs) - search space를 탐색 후 계산 가능한 범위로 줄여야 하기 때문이다
# link를 어떻게 찾을 것인가? => link 생긴게 지 멋대로 생겨있음 (ex)mailto:...) => 따라서 정규화 실행 => 다음번 방문할 link (방문한곳, 방문할 곳에 포함되지 않은 link)
# 이것을 focused crawling으로 link 방문을 제한시켜서 사용 가능하게끔 만듦
# => focused crawling + scraping + DB
# => 이때 도는 link를 우선순위로 만듦 Citation(href:linking) => PageRank 라는 알고리즘으로 Citation 우선순위를 매길 수 있다

In [4]:
from requests.compat import urlparse, urljoin
from requests import get
import re

def canFetch(url, path):
    resp = get(urljoin(url, '/robots.txt')) # 특정 사이트 robot.txt 접근
    if resp.status_code != 200: # 200 OK만 나오면 괜찮
        return True
    
    disallowList = [link for link in re.findall(r'disallow\s*:\s*(.+)', resp.text, re.IGNORECASE)]
    # 대문자 소문자 무시하고, resp.text 중에서 disallow 뒤에 있는 애들을 link라는 for문으로 list 작성
    
    if urlparse(path).path in disallowList:
        return False
    # 만약 path가 disallowList(즉, 가면 안되는 곳)에 있으면 False 반환

    return True # 나머지는 True 반환

canFetch('https://www.google.com', '/search?q=수지')

False

In [7]:
from requests import request
from requests.exceptions import HTTPError
from time import sleep

def download(url, params={}, data={}, headers={}, method = 'GET', retries=3):
    # if not canFetch(url, url):  # robots.txt 검사
    #     print('수집하면 안됨')  # False일때

    resp = request(method, url, params=params, data=data, headers=headers)
    # parameter로 받은 친구들을 request 옵션으로 넣어서 request 객체 생성
    try:
        resp.raise_for_status() # 에러 확인 용도, HTTPError 뜨면 except로 동작
    except HTTPError as e:
        if 499 < resp.status_code and retries > 0:  # 500번 에러 그리고 retry가 0이 아직 아닐때
            print('재시도 중')                       # 계속 돌아야 하기 때문임
            sleep(5)                                # traffic 너무 먹으면 안되므로
            return download(url, params, data, headers, method, retries-1)  # retry 한번 했으니 하나 빼고
        else:
            print(e.response.status_code)
            print(e.request.headers)
            print(e.response.headers)
            # 에러들 보여주기
            return None
    
    return resp
    # 마지막으로 request 객체 반환
# download('https://httpbin.org/status/500')

In [ ]:
from bs4 import BeautifulSoup

URLs = []
Visited = []
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

# URLs.appen('https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지')
URLs.append('https://www.google.com/search?q=수지')

while URLs:
    url = URLs.pop(0)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)

    Visited.append(url)

    resp = download(url, headers={'user-agent': ua})
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url, newURL)
                if noremlizedURL not in Visited and noremlizedURL not in URLs:
                    URLs.append(noremlizedURL)

    # print(len(URLs), len(Visited))

In [ ]:
# focused crawling 만들어보자
# 내 관심사에만 집중하자
# 전략 존재
# 1. depth제한 - depth가 깊어질수록 원래 관심사에서 멀어지는 경향이 존재하기 때문이다
# 2. domian 제한 - 볼 필요 없는 블로그 혹은 카페 이런 것들을 제외시키는 것
# 3. HTML tags 제한

URLs = []
Visited = []
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0}) # fix
# URLs.append('https://www.google.com/search?q=수지')

while URLs:
    url = URLs.pop(0)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)

    Visited.append(url['link']) # fix

    resp = download(url['link'], headers={'user-agent': ua}) # fix
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)            # fix
                if noremlizedURL not in Visited and noremlizedURL not in URLs:
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1}) # fix

    # print(len(URLs), len(Visited))

In [12]:
print(len(URLs), len(Visited)), Visited[-1], URLs[-1]

629 13


(None,
 'https://dict.naver.com/dict.search?query=%EC%88%98%EC%A7%80&from=tsearch',
 {'link': 'https://www.navercorp.com/', 'depth': 2})

In [ ]:
# stack 변환
# 원래 정보에서 굉장히 많이 떨어진 페이지가 나온다

URLs = []
Visited = []
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0}) # fix
# URLs.append('https://www.google.com/search?q=수지')

while URLs:
    url = URLs.pop(-1)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)

    Visited.append(url['link']) # fix

    resp = download(url['link'], headers={'user-agent': ua}) # fix
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)            # fix
                if noremlizedURL not in Visited and noremlizedURL not in URLs:
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1}) # fix

In [14]:
print(len(URLs), len(Visited)), Visited[-1], URLs[-1]

904 8


(None,
 'https://www.youtube.com/t/terms?archive=20191210',
 {'link': 'https://support.google.com/accounts/contact/suspended?p=youtube',
  'depth': 7})

In [ ]:
URLs = []
Visited = []
Skipped = []    # fix
ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 3   # 제한할 depth  # fix

URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0}) # fix
# URLs.append('https://www.google.com/search?q=수지')

while URLs:
    url = URLs.pop(-1)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)
    
    if url['depth'] > depth:    # 제한 1. 깊이로 제한
        Skipped.append(url['link']) # fix
        continue

    Visited.append(url['link'])

    resp = download(url['link'], headers={'user-agent': ua}) 
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)            
                if noremlizedURL not in Visited and noremlizedURL not in URLs and noremlizedURL not in Skipped: # fix
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

In [17]:
print(len(URLs), len(Visited)), len(Skipped), Visited[-1], URLs[-1], Skipped[-1]

657 9


(None,
 279,
 'https://www.youtube.com/t/privacy',
 {'link': 'https://www.youtube.com/t/terms', 'depth': 3},
 'https://www.youtube.com/howyoutubeworks/static/images/eyebrow-underline.png')

In [ ]:
# 너비 우선 탐색은 메모리를 많이 먹음
# 깊이 우선 탐색은 모든걸 기록해놔야함
# space complexity가 높아질수 밖에 없다

from requests.compat import urlparse

# 2. 도메인 제한
URLs = []
Visited = []
Skipped = []

optin = ['v.daum.net']  # 여기 있는 것들만
optout = [] # 여기 제외

ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 3   # 제한할 depth  # fix

# URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0})
# URLs.append('https://www.google.com/search?q=수지')
URLs.append({'link':'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=수지', 'depth': 0}) # fix

while URLs:
    url = URLs.pop(-1)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)
    
    if url['depth'] > depth:    # 제한 1. 깊이로 제한
        Skipped.append(url['link']) # fix
        continue

    Visited.append(url['link'])

    resp = download(url['link'], headers={'user-agent': ua}) 
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)
                # 2. 도메인 제한 -optin
                # if urlparse(noremlizedURL).netloc not in optin:
                #     continue
                # 해당 패스하는 애들만 아래에 저장 
                
                # 3. 도메인 제한 -optout
                # if urlparse(noremlizedURL).netloc in optout:
                #     continue
                # 해당 맞는 애들은 거르기
                if noremlizedURL not in Visited and noremlizedURL not in URLs and noremlizedURL not in Skipped: # fix
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

In [20]:
print(len(URLs), len(Visited), len(Skipped)), Visited[-1], URLs[-1], Skipped[-1]

IndexError: list index out of range

In [ ]:
# naver blog만 해보자
# iframe - dom을 dom 안에 넣는 방식

URLs = []
Visited = []
Skipped = []

optin = ['blog.naver.com']  # 여기 있는 것들만
optout = [] # 여기 제외

ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 3   # 제한할 depth  # fix

URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0})
# URLs.append('https://www.google.com/search?q=수지')
# URLs.append({'link':'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=수지', 'depth': 0}) # fix

while URLs:
    url = URLs.pop(-1)   # 0: FIFO; Queue / -1: FILO; Stack ==> url = URLs.pop(-1)
    
    if url['depth'] > depth:    # 제한 1. 깊이로 제한
        Skipped.append(url['link']) # fix
        continue

    Visited.append(url['link'])

    resp = download(url['link'], headers={'user-agent': ua}) 
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')

        for link in dom.select('a[href], iframe[src], img[src], audio[src], frame[src], video[src], style[src], link[src]'):
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)
                # 2. 도메인 제한 -optin
                if urlparse(noremlizedURL).netloc not in optin:
                    continue
                # 해당 패스하는 애들만 아래에 저장 
                
                # 3. 도메인 제한 -optout
                # if urlparse(noremlizedURL).netloc in optout:
                #     continue
                # 해당 맞는 애들은 거르기
                if noremlizedURL not in Visited and noremlizedURL not in URLs and noremlizedURL not in Skipped: # fix
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

In [22]:
print(len(URLs), len(Visited), len(Skipped)), Visited[-1], URLs[-1], Skipped[-1]

50 8 108


(None,
 'https://blog.naver.com/PostList.naver?blogId=travis88&categoryNo=4&from=postList&parentCategoryNo=0',
 {'link': 'https://blog.naver.com/travis88/223305078411', 'depth': 3},
 'https://blog.naver.com')

In [23]:
Visited     # 방문한 블로그들

['https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지',
 'https://blog.naver.com/travis88/223325700661',
 'https://blog.naver.com/PostView.naver?blogId=travis88&logNo=223325700661&redirect=Dlog&widgetTypeCall=true&directAccess=false',
 'https://blog.naver.com/FILEPATH',
 'https://blog.naver.com/FILEPATH',
 'https://blog.naver.com/connect/WidgetView.naver?blogId=travis88&widgetSeq=1&authCode=be74f3a542918ab9c904c5354b8a691a6957adf',
 'https://blog.naver.com/PostList.naver?blogId=travis88&categoryNo=4&from=postList&parentCategoryNo=0',
 'https://blog.naver.com/PostList.naver?blogId=travis88&categoryNo=4&from=postList&parentCategoryNo=0']

In [28]:
# optin, optout 둘 다 더 추가해서 제한 가능하다
# 경로를 제한 가능하다는 뜻
# 이번엔 뉴스 링크만

URLs = []
Visited = []
Skipped = []

optin = ['news.naver.com', 'm.news.naver.com']  # news만
optout = []

ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 5   

# URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0})
# URLs.append('https://www.google.com/search?q=수지')
# URLs.append({'link':'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=수지', 'depth': 0}) # fix
URLs.append({'link':'https://news.naver.com/', 'depth': 0})

while URLs:
    url = URLs.pop(0) 
    
    if url['depth'] > depth: 
        Skipped.append(url['link'])
        continue

    Visited.append(url['link'])

    resp = download(url['link'], headers={'user-agent': ua}) 
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        # 3. 영역제한 - 뉴스만

        # 뉴스인 경우
        # 3-1. 메뉴
        linksA = dom.select('ul.nNlnb_menu_list li > a') # ul 클래스 가진 li의 자식인 a를 찾음  # fix
        # 위의 부모가 .sa|ss_text > a:has(> strong) sa혹은 ss_text 클래스가 a를 가지는데 이 a는 strong을 가진다
        # 3-2.뉴스
        linksB = dom.select('div[class$="_text"] > a[href]:has(>strong)')   # fix

        for link in linksA+linksB:  # fix
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)
                if urlparse(noremlizedURL).netloc not in optin:
                    continue
                
                # if urlparse(noremlizedURL).netloc in optout:
                #     continue
                if noremlizedURL not in Visited and noremlizedURL not in URLs and noremlizedURL not in Skipped: # fix
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

# 맨처음에 menu만 들어가므로 linksB가 비어있을 것임
# 그 후 어딘가에 섹션에 들어가면 linksB가 나올 것이다

In [29]:
len(URLs), len(Visited), len(Skipped), Visited[-1], #URLs[-1]

(0, 1, 0, 'https://news.naver.com/')

In [30]:
# 따라서, focused crawling은 뚜렷한 목적에 따라서 수집은 하지만 고정된 것은 아니다
# scraping도 섞자
from os import mkdir, listdir

In [31]:
listdir('.')
# 현재 작업중인 폴더의 파일들

['data_collect_4.ipynb']

In [32]:
mkdir('./naver_news')
# 생성 완료

In [65]:
# news 본문만 저장

URLs = []
Visited = []
Skipped = []

optin = ['news.naver.com', 'n.news.naver.com']  # news만
optout = []

ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 5   

# URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0})
# URLs.append('https://www.google.com/search?q=수지')
# URLs.append({'link':'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=수지', 'depth': 0}) # fix
URLs.append({'link':'https://news.naver.com/', 'depth': 0})

NEWS_TEXT = './naver_news'
NEWS_IMG = './naver_img'

while URLs:
    url = URLs.pop(-1) 
    
    if url['depth'] > depth: 
        Skipped.append(url['link'])
        continue

    resp = download(url['link'], headers={'user-agent': ua})

    Visited.append(url['link'])
    
    if resp is None:
        continue

    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        # 3. 영역제한 - 뉴스만

        # 뉴스인 경우
        # 3-1. 메뉴
        linksA = dom.select('ul.Nlnb_menu_list li > a') # ul 클래스 가진 li의 자식인 a를 찾음  # fix
        # 위의 부모가 .sa|ss_text > a:has(> strong) sa혹은 ss_text 클래스가 a를 가지는데 이 a는 strong을 가진다
        # 3-2.뉴스
        linksB = dom.select('div[class$="_text"] > a[href]:has(> strong)')   # fix

        # 3-3. 뉴스 본문
        linksC = dom.select('#dic_area a[href]')
        article = dom.select_one('#dic_area')
        # 뉴스 본문을 찾기 위해 특정 element만

        if article:
            with open(NEWS_TEXT+'/'+re.search(r'(\d{10})$',url['link']).group(1)+'.txt', 'w', encoding='utf8') as fp:  # 기사 number로 text 파일 이름 줄것임
                fp.write(article.get_text())
        # 저장 file

        for link in linksA+linksB+linksC:  # fix
            newURL = link.attrs['href'] if link.has_attr('href') else link.attrs['src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)
                if urlparse(noremlizedURL).netloc not in optin:
                    continue
                
                # if urlparse(noremlizedURL).netloc in optout:
                #     continue
                if noremlizedURL not in Visited and noremlizedURL not in URLs and noremlizedURL not in Skipped: # fix
                    # fix url['link'] for url in URLs
                    # why? dict인데 item 안에 들어있으니 in으로 체크하면 무조건 True가 나올 것임
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

KeyboardInterrupt: 

In [67]:
len(URLs), len(Visited), len(Skipped), Visited[-1], #URLs[-1]
Visited

['https://news.naver.com/',
 'https://news.naver.com/main/ombudsman/errorArticleList.naver?mid=omb',
 'https://news.naver.com/factcheck/main',
 'https://news.naver.com/main/tv/index.naver?mid=tvh',
 'https://news.naver.com/opinion/home',
 'https://news.naver.com/newspaper/home?viewType=pc',
 'https://news.naver.com/main/ranking/popularDay.naver',
 'https://news.naver.com/section/104',
 'https://n.news.naver.com/mnews/article/421/0007413124',
 'https://n.news.naver.com/mnews/article/003/0012428296',
 'https://n.news.naver.com/mnews/article/016/0002280804',
 'https://n.news.naver.com/mnews/article/277/0005392558',
 'https://n.news.naver.com/mnews/article/003/0012428375',
 'https://n.news.naver.com/mnews/article/277/0005392570',
 'https://n.news.naver.com/mnews/article/003/0012428399',
 'https://n.news.naver.com/mnews/article/015/0004960267',
 'https://n.news.naver.com/mnews/article/011/0004313593',
 'https://n.news.naver.com/mnews/article/022/0003914302']

In [93]:
# news 본문만 저장

URLs = []
Visited = []
Skipped = []

optin = ['news.naver.com', 'n.news.naver.com', 'imgnews.pstatic.net']  # news만
optout = []

ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'

depth = 5   

# URLs.append({'link':'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=수지', 'depth': 0})
# URLs.append('https://www.google.com/search?q=수지')
# URLs.append({'link':'https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=수지', 'depth': 0}) # fix
URLs.append({'link':'https://news.naver.com/', 'depth': 0})

NEWS_TEXT = './naver_news'
NEWS_IMG = './naver_img'

while URLs:
    url = URLs.pop(-1) 
    
    if url['depth'] > depth: 
        Skipped.append(url['link'])
        continue

    resp = download(url['link'], headers={'user-agent': ua})

    Visited.append(url['link'])
    
    if resp is None:
        continue

    if re.search(r'image\/(\w+);?', resp.headers['content-type']):
        ext = re.search(r'image\/(\w+);?', resp.headers['content-type']).group(1)
        fname = re.search(r'/(\w+\.jpg|jpeg|png|bmp|gif)', url['link'])
        if fname:    
            with open(NEWS_IMG+'/'+fname.group(1), 'wb') as fp:
                fp.write(resp.content)
            # binary 값이므로 resp.content
        else:
            print(url['link'], resp.headers, ext)
    if re.search(r'text\/html', resp.headers['content-type']):
        dom = BeautifulSoup(resp.text, 'html.parser')
        # 3. 영역제한 - 뉴스만

        # 뉴스인 경우
        # 3-1. 메뉴
        linksA = dom.select('ul.Nlnb_menu_list li > a') # ul 클래스 가진 li의 자식인 a를 찾음  # fix
        # 위의 부모가 .sa|ss_text > a:has(> strong) sa혹은 ss_text 클래스가 a를 가지는데 이 a는 strong을 가진다
        # 3-2.뉴스
        linksB = dom.select('div[class$="_text"] > a[href]:has(> strong)')   # fix

        # 3-3. 뉴스 본문
        linksC = dom.select('#dic_area a[href], #dic_area img[data-src]')   # 여기 바꿈 naver는 data-src이기 떄문에
        article = dom.select_one('#dic_area')
        # 뉴스 본문을 찾기 위해 특정 element만

        if article:
            with open(NEWS_TEXT+'/'+re.search(r'(\d{10})$',url['link']).group(1)+'.txt', 'w', encoding='utf8') as fp:  # 기사 number로 text 파일 이름 줄것임
                fp.write(article.get_text())
        # 저장 file

        for link in linksA+linksB+linksC:  # fix
            if link.has_attr('href'):
                newURL = link.attrs['href']
            elif link.has_attr('src'):
                newURL = link.attrs['src']
            elif link.has_attr('data-src'):
                newURL = link.attrs['data-src']

            if not re.match(r'#|javascript|mailto|data', newURL):
                noremlizedURL = urljoin(url['link'], newURL)
                if urlparse(noremlizedURL).netloc not in optin:
                    continue
                
                # if urlparse(noremlizedURL).netloc in optout:
                #     continue
                if noremlizedURL not in Visited and noremlizedURL not in [url['link'] for url in URLs] and noremlizedURL not in Skipped: # fix
                    URLs.append({'link':noremlizedURL, 'depth': url['depth']+1})

KeyboardInterrupt: 

In [73]:
resp = download('https://imgnews.pstatic.net/image/030/2024/03/15/0003189376_001_20240315091501054.jpg?type=w647')
re.search(r'image\/(\w+);?', resp.headers['content-type'])
# jpg 맞는지 확인

<re.Match object; span=(0, 10), match='image/jpeg'>

In [76]:
re.search(r'/(\w+\.jpg|jpeg|png|bmp|gif)', 'https://imgnews.pstatic.net/image/030/2024/03/15/0003189376_001_20240315091501054.jpg?type=w647').groups(1)
# 파일 이름 구하기

('0003189376_001_20240315091501054.jpg',)

In [92]:
fname = re.search(r'/(\w+\.jpg|jpeg|png|bmp|gif)', 'https://imgnews.pstatic.net/image/030/2024/03/15/0003189376_001_20240315091501054.jpg?type=w647').group(1)
print(fname)
resp = download(url['link'], headers={'user-agent': ua})
print(url['link'], resp.headers)

0003189376_001_20240315091501054.jpg
https://imgnews.pstatic.net/image/277/2024/03/15/0005392596_001_20240315102601385.jpg?type=w647 {'Accept-Ranges': 'bytes', 'Content-Length': '198962', 'Content-Type': 'image/jpeg', 'Last-Modified': 'Fri, 15 Mar 2024 01:26:19 GMT', 'P3p': 'CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"', 'Cache-Control': 'max-age=6702', 'Expires': 'Fri, 15 Mar 2024 05:19:38 GMT', 'Date': 'Fri, 15 Mar 2024 03:27:56 GMT', 'Connection': 'keep-alive'}


In [ ]:
# crawling이랑 scraping이랑 다르다
# 많이들 혼동해서 사용한다
# 하지만 분명히 다르다
# crawling - 전체 html 파일이 필요한다 / large scale
# scraping - 특정 url에서 링크 가져오고 이름 가져오고 등등 / not large scale - 즉, 내가 원하는 부분만 가능 / 원할때 그만할 수 있음
# focused - large scale / 끝없이 나오기 때문에
# 이제는 scraping을 사용할 것이다